## Week 4: Supervised Learning III
### Neural Networks
So, in this week we will be using pytorch, one of the best libraries for neural networks.
* To apply the perceptron and multilayer perceptron from Week 4 lecture to automatic detection of the number of days of ground frost and snow based on other weather variables.


To achieve the above goal, we'll need first to seperate the dataset into training and testing. We'll apply 70:30 split and then further split testing into validation:test as 50:50.

We'll first import the csv file, read it and then apply seperation

In [ ]:
import numpy as np
import csv

file_path = "/Users/suli/Documents/source/repo/MachineLearning/Week 4/curated_data_1month_2010-2022_nonans.csv"

data_list = []
with open(file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    row_count = 0
    for row in reader:
        if row_count > 0:
            data_list.append(row)
        row_count += 1
        
data = np.array(data_list)
print(data.shape)
print(data[0])

feat_col = [5, 6, 7, 8, 9, 10, 12]
ground_frost_col = 4
snow_col = 11 # should be 11 for snow and 12 for rain

feats = data[:, feat_col]
ground_frost_label = data[:, ground_frost_col]
snow_label = data[:, snow_col]

print("\n A peek at the dataset features: \n"+str(feats))
print("\n A peek at the ground frost labels: \n"+str(ground_frost_label))
print("\n A peek at the snow labels: \n"+str(snow_label))

(10297, 13)
['x_coord' 'y_coord' 'month' 'year' 'groundfrost_1' 'hurs_1' 'psl_1'
 'sun_1' 'pv_1' 'sfcWind_1' 'tas_1' 'snowLying_1' 'rainfall_1']

 A peek at the dataset features: 
[['hurs_1' 'psl_1' 'sun_1' ... 'sfcWind_1' 'tas_1' 'rainfall_1']
 ['89.36982749' '1022.665365' '64.51156417' ... '6.45810733'
  '6.727447717' '112.2352382']
 ['89.44621093' '1022.708003' '57.48681167' ... '5.881910517'
  '6.230648281' '116.3547495']
 ...
 ['87.83702931' '1006.457057' '13.8800195' ... '4.960640256'
  '1.856263012' '177.2424627']
 ['88.81163154' '1006.622483' '20.58531624' ... '4.936354968'
  '0.775835354' '135.4028786']
 ['82.76015165' '1005.938301' '10.53091935' ... '8.380819417'
  '3.545097582' '140.831213']]

 A peek at the ground frost labels: 
['groundfrost_1' '9.84928987' '10.85267889' ... '21.7275541' '23.77582838'
 '17.35386163']

 A peek at the snow labels: 
['snowLying_1' '0.697199793' '1.629525681' ... '7.997095434' '8.468158997'
 '6.3599061']


Right, so now we will be working on splitting into training and validation and tests

In [18]:
from sklearn.model_selection import train_test_split

all_ids = np.arange(feats.shape[0])
print(all_ids)

random_seed =1

train_set_id, rem_set_id = train_test_split(all_ids, test_size=0.3, train_size=0.7, random_state=random_seed, shuffle=True)

validate_set_id, test_set_id = train_test_split(rem_set_id, test_size=0.5, train_size=0.5, random_state=random_seed, shuffle=True)

train_set_data = feats[train_set_id,:]
train_ground_frost_labels = ground_frost_label[train_set_id]
train_snow_labels = snow_label[train_set_id]

validate_set_data = feats[validate_set_id,:]
validate_ground_frost_labels = ground_frost_label[validate_set_id]
validate_snow_labels = snow_label[validate_set_id]

test_set_data = feats[test_set_id,:]
test_ground_frost_labels = ground_frost_label[test_set_id]
test_snow_labels = snow_label[test_set_id]



[    0     1     2 ... 10294 10295 10296]


Perfect, now we have all of our data ready but before moving to the next step. it's better to scale our training data such that all datapoints are on a 0-1 scale. This makes our neural network perform better. Simply because If one feature has values in the range of [0,1], and another feature is in the range of [1000,5000], the model may assign more importance to the larger values.


In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(feats)
scaled_feats = scaler.transform(feats)
print("\n A peek at the scaled dataset features: \n"+str(scaled_feats))


scaled_train_data = scaled_feats[train_set_id, :]
scaled_val_data = scaled_feats[validate_set_id, :]
scaled_test_data = scaled_feats[test_set_id, :]

ValueError: could not convert string to float: np.str_('hurs_1')